In [8]:
from pathlib import Path

data_path = Path("data/")
image_path = data_path / "events"
train_dir = image_path / "train_scaled"
test_dir = image_path / "test_scaled"

train_dir, test_dir
output_dir = 'output'

In [9]:

import torch
torch.cuda.empty_cache()
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [10]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)

In [11]:
import matplotlib.pyplot as plt
import torch

from torch import nn
from torchvision import transforms

from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from torchvision.models import ResNet152_Weights, resnet152

weights = EfficientNet_V2_S_Weights.DEFAULT
model = efficientnet_v2_s(weights=weights).to(device)
# weights = ResNet152_Weights.DEFAULT
# model = resnet152(weights=weights).to(device)



In [12]:
weights.transforms()

ImageClassification(
    crop_size=[384]
    resize_size=[384]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [13]:
auto_transforms = weights.transforms()
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform = transforms.Compose([
    # Resize the input image to a square of size 224x224
    transforms.Resize((428, 428)),

    # Apply a random horizontal flip to augment the data
    transforms.RandomHorizontalFlip(),

    # Convert the image to a PyTorch tensor
    transforms.ToTensor(),

    # Normalize the image with mean and standard deviation values
    # These values are typical for the ImageNet dataset, but you might
    # need to adjust them based on your specific dataset
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [14]:
from torchvision import datasets
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=auto_transforms, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)
test_data = datasets.ImageFolder(root=test_dir,
                                 transform=auto_transforms)
# train_data = datasets.ImageFolder(root=train_dir, # target folder of images
#                                   transform=train_transform, # transforms to perform on data (images)
#                                   target_transform=None) # transforms to perform on labels (if necessary)
# test_data = datasets.ImageFolder(root=test_dir,
#                                  transform=val_transform)


In [15]:

class_names = train_data.classes
class_names

['Combat',
 'DestroyedBuildings',
 'Fire',
 'Humanitarian Aid and rehabilitation',
 'Military vehicles and weapons']

In [16]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=16,shuffle=False)

In [17]:
# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)
# torch.autograd.set_detect_anomaly(True)
# Recreate the classifier layer and seed it to the target device

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=output_shape, bias=True),
    # nn.ReLU6(),
).to(device)

In [18]:
from torch import nn
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1) # this is also called "criterion"/"cost function" in some places
# optimizer = torch.optim.RMSprop(params=model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

In [19]:
torch.cuda.set_device(0)

In [20]:
next(model.parameters()).is_cuda

True

In [21]:
# Set the random seeds
try:
    from going_modular.going_modular import engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !move pytorch-deep-learning\going_modular .
    !rmdir /s /q pytorch-deep-learning
    from going_modular.going_modular import engine

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=15,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.4348 | train_acc: 0.5000 | test_loss: 1.0746 | test_acc: 0.8625
Epoch: 2 | train_loss: 0.7942 | train_acc: 0.8812 | test_loss: 0.6058 | test_acc: 0.9125
Epoch: 3 | train_loss: 0.5580 | train_acc: 0.9344 | test_loss: 0.5324 | test_acc: 0.9250
Epoch: 4 | train_loss: 0.4700 | train_acc: 0.9812 | test_loss: 0.5331 | test_acc: 0.9250
Epoch: 5 | train_loss: 0.4524 | train_acc: 0.9938 | test_loss: 0.5282 | test_acc: 0.9500
Epoch: 6 | train_loss: 0.4479 | train_acc: 0.9969 | test_loss: 0.5173 | test_acc: 0.9625
Epoch: 7 | train_loss: 0.4377 | train_acc: 0.9906 | test_loss: 0.5127 | test_acc: 0.9375
Epoch: 8 | train_loss: 0.4386 | train_acc: 0.9969 | test_loss: 0.4979 | test_acc: 0.9750
Epoch: 9 | train_loss: 0.4342 | train_acc: 0.9969 | test_loss: 0.4980 | test_acc: 0.9500
Epoch: 10 | train_loss: 0.4155 | train_acc: 1.0000 | test_loss: 0.5112 | test_acc: 0.9625
Epoch: 11 | train_loss: 0.4227 | train_acc: 1.0000 | test_loss: 0.4880 | test_acc: 0.9625
Epoch: 12 | train_l

In [22]:
torch.save(model, 'model.tf')
torch.save(model.state_dict(), 'weights.tf')